In [43]:
import pandahouse as ph
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from scipy.stats import norm, ttest_ind
from tqdm import tqdm

In [44]:
rng = np.random.default_rng()  # Объект для рандомизации из numpy

In [45]:
#Распределение просмотров
connection = {'host': 'https://clickhouse.lab.karpov.courses',
'database':'simulator_20250520',
'user':'student',
'password':'dpo_python_2020'
}
## АА-тест
q = """
select views, count() as users
  from (select user_id,
               sum(action = 'view') as views
          from {db}.feed_actions
  where toDate(time) between '2025-04-25' and '2025-05-01'
  group by user_id
)
group by views
order by views
"""

views_distribution = ph.read_clickhouse(q, connection=connection)

In [46]:
views_distribution['p'] = views_distribution['users']/views_distribution.users.sum()
views_distribution.sort_values(by = 'p', ascending = False)

,views,users,p
15,16,545,0.012977
14,15,537,0.012787
13,14,500,0.011906
34,35,485,0.011548
29,30,469,0.011167
...,...,...,...
296,324,1,0.000024
297,326,1,0.000024
298,355,1,0.000024
299,364,1,0.000024


In [47]:
#Распределение ctr
q = """
select floor(ctr, 2) as ctr, count() as users
  from (select toDate(time) as dt, 
               user_id,
               sum(action = 'like')/sum(action = 'view') as ctr
          from {db}.feed_actions
         where dt between '2025-04-25' and '2025-05-01'
         group by dt, user_id
        )
 group by ctr
"""


ctr_distribution = ph.read_clickhouse(q, connection=connection)
ctr_distribution['p'] = ctr_distribution['users']/ctr_distribution.users.sum()

#### Сколько пользователей приходится на одну группу

In [48]:
q = """
SELECT COUNT(DISTINCT user_id)
  FROM   {db}.feed_actions 
 WHERE toDate(time) between '2025-04-25' and '2025-05-01'
"""

df = ph.read_clickhouse(q, connection=connection) # Неделя AA-теста

In [49]:
q = """
SELECT COUNT(DISTINCT user_id)
  FROM   {db}.feed_actions 
 WHERE toDate(time) between '2025-04-25' and '2025-05-08'
"""

df = ph.read_clickhouse(q, connection=connection) # 1 неделя AA-теста, 1 неделя AB-теста

In [50]:
usrs = round(df.iloc[0, 0]/2, 0).astype('int32') # кол-во пользователей за 2 недели

In [51]:
usrs

np.int32(30591)

#### Оценка начального размера выборки 

In [60]:
sample_size = usrs
exp_size = 20000 # Количество симуляций

Модифицированные просмотры формируются следующим образом. Сначала биномальным распределением иммитируется добавление 1 или 2 просмотров, причем с события предполагаются равновероятными p=0.5. ```np.binomial(n=1, p=0.5, size=размер_выборки)``` равновероятно принимает значение 0 или 1, при добавлении 1 получаем равновероятное добавление 1 или 2 просмотров. Далее по правилу перемножения вероятности иммитируется, что алгоритм сработает в 90% случаев. А именно домножается на соответвуещее биномиальное распределение с исходами сработал 1 и не сработал 0. Далее все умножается на глобальное условие, что алгоритм вообще не будет работать если в группе B будет меньше 50 просмотров.

In [65]:
p_values = []
for _ in tqdm(range(exp_size)):
    #Просмотры в группе А
    A_views = rng.choice(
        views_distribution['views'], size=sample_size, 
        replace=True, p=views_distribution['p']).astype("int32")
    #Просмотры в группе B 
    B_views = rng.choice(
        views_distribution['views'], size=sample_size, 
        replace=True, p=views_distribution['p']).astype("int32")
    #Модифицирование просмотров в группе B
    B_views = B_views + ((1 + rng.binomial(n=1, p=0.5, size=sample_size)) 
        * rng.binomial(n=1, p=0.9, size=sample_size) 
        * (B_views >= 30))

    group_A_views = np.array(A_views)
    group_B_views = np.array(B_views)

    #CTR в группе А
    A_ctr = rng.choice(
        ctr_distribution['ctr'], size=sample_size, 
        replace=True, p=ctr_distribution['p'])
    #CTR в группе B
    B_ctr = rng.choice(
        ctr_distribution['ctr'], size=sample_size, 
        replace=True, p=ctr_distribution['p'])

    group_A_ctr = np.array(A_ctr)
    group_B_ctr = np.array(B_ctr)

    #Clicks в группе А
    A_clicks = rng.binomial(
        n=group_A_views, p=group_A_ctr, size=sample_size)
    #CTR в группе B
    B_clicks = rng.binomial(
        n=group_B_views, p=group_B_ctr, size=sample_size)

    group_A_clicks = np.array(A_clicks)
    group_B_clicks = np.array(B_clicks)

    # Будем отбирать только пользователей c 30 и больше просмотров
    # Так как алгоритм не затрагивает у кого просмотров меньше 30
    
    mask_A = group_A_views >= 30
    mask_B = group_B_views >= 30

    masked_A_clicks = group_A_clicks[mask_A]
    masked_B_clicks = group_B_clicks[mask_B]

    p_values.append(ttest_ind(masked_A_clicks, masked_B_clicks, equal_var=False).pvalue)


100%|██████████| 20000/20000 [04:55<00:00, 67.79it/s]


In [66]:
p_values = np.array(p_values)

In [67]:
np.sum(p_values <= 0.05)/exp_size # Мощность теста

np.float64(0.6491)